In [2]:
import os
import numpy as np
import pandas as pd
import datetime

subs = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'D.C.': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [3]:
# Read in JHU Data
JHU_data =  pd.read_csv('../JHU dataset/JHU_filtered_timeseries.csv')

In [4]:
locs_np = np.array(JHU_data['Province/State'])

states_list = []
counties_list = []

for i in np.arange(0,len(locs_np)):
    loc = locs_np[i]
    both = loc.split(',')
    county = both[0].rstrip().lstrip()
    state = both[1].rstrip().lstrip()
    states_list.append(state)
    counties_list.append(county)

In [5]:
states = pd.DataFrame(states_list, columns=['State'])
counties = pd.DataFrame(counties_list, columns=['Province'])

locations = counties.join(states)
cleaned = locations.join(JHU_data)
cleaned = cleaned.drop(columns=['Province/State'])
cleaned = cleaned.rename(columns={'Country/Region': 'Country'})

In [6]:
d24 = pd.read_csv('../Daily Updates/Mar 24.csv')
d25 = pd.read_csv('../Daily Updates/Mar 25.csv')

In [7]:
long_states = list(d25.State)
rev_subs = { v:k for k,v in subs.items()}
short_states = [rev_subs.get(item,item)  for item in long_states]

In [9]:

def COUNTY_STATE(county0,state0):
    '''
    Converts county and state to combined version.
    This reduces confusion between strings with similar meanings
    
    Input:  'county', 'state'
    Return:  'COUNTY,STATE' (without spaces, - , or /)
    
    
    Examples of same meaning, different spellings:  
    - Washington, D.C.   &   Washington, DC
    - Valdez-Cordova, AK & Valdez Cordova, AK
    '''
    # uppercase
    county1 = county0.upper()
    state1 = state0.upper()
    # remove trailing/leading spaces and periods and dashes
    county2 = county1.replace(' ','').replace('.','').replace('/','').replace('-','').replace('COUNTY','').replace('\'','').replace('PARISH','')
    state2 = state1.replace(' ','').replace('.','').replace('/','').replace('-','').replace('COUNTY','').replace('\'','').replace('PARISH','')
    # combine for key
    string = county2+','+state2
    return string



##############################################################################
# create FIPS array with unique and distinguishable county/state tags
#
#  -this eliminates problems associated with capital letters and symbols
##############################################################################

fips = pd.read_csv('../FIPS_tags/FIPS_county_state.csv')
fips_np = np.array(fips)
name_list = []

for i in np.arange(0,len(fips_np)):
    string = COUNTY_STATE(fips_np[i,1],fips_np[i,2])
    name_list.append(string)

fips_names = np.column_stack((fips_np[:,0],np.array(name_list)))
code_caps = pd.DataFrame(fips_names, columns=['FIPS code','COUNTY,STATE'])

############ Save to CSV file.  Previously unhashed ################
code_caps.to_csv('../FIPS_tags/code_COUNTYSTATE.csv',index=False)

In [7]:
import os
import numpy as np
import pandas as pd
import datetime

########################################################################

def COUNTY_STATE(county0,state0):
    '''
    Converts county and state to combined version.
    This reduces confusion between strings with similar meanings
    
    Input:  'county', 'state'
    Return:  'COUNTY,STATE' (without spaces, - , or /)
    
    
    Examples of same meaning, different spellings:  
    - Washington, D.C.   &   Washington, DC
    - Valdez-Cordova, AK & Valdez Cordova, AK
    '''
    # uppercase
    county1 = county0.upper()
    state1 = state0.upper()
    # remove trailing/leading spaces and periods and dashes
    county2 = county1.replace(' ','').replace('.','').replace('/','').replace('-','').replace('COUNTY','').replace('\'','').replace('PARISH','')
    state2 = state1.replace(' ','').replace('.','').replace('/','').replace('-','').replace('COUNTY','').replace('\'','').replace('PARISH','')
    # combine for key
    string = county2+','+state2
    return string



##############################################################################
# create FIPS array with unique and distinguishable county/state tags
#
#  -this eliminates problems associated with capital letters and symbols
##############################################################################

fips = pd.read_csv('../FIPS_tags/FIPS_county_state.csv')
fips_np = np.array(fips)
name_list = []

for i in np.arange(0,len(fips_np)):
    string = COUNTY_STATE(fips_np[i,1],fips_np[i,2])
    name_list.append(string)

fips_names = np.column_stack((fips_np[:,0],np.array(name_list)))
code_caps = pd.DataFrame(fips_names, columns=['FIPS code','COUNTY,STATE'])

############ Save to CSV file.  Previously unhashed ################
#code_caps.to_csv('../FIPS_tags/code_COUNTYSTATE.csv',index=False)

##############################################################################
# Analyze cencus data to find GPS coordinates for counties
##############################################################################



census = np.array(pd.read_csv('../FIPS_tags/cbg_geographic_data.csv'))
fips_long = census[:,0].astype(str)
# Convert latitudes and longitudes to radians
lat = np.deg2rad(census[:,-2])
lon = np.deg2rad(census[:,-1])
#Convert lat/lon (must be in radians) to Cartesian coordinates for each location.
X = np.cos(lat) * np.cos(lon)
Y = np.cos(lat) * np.sin(lon)
Z = np.sin(lat)


def avg_loc(code):
    code = str(code)
    lcode = int(len(code))
    cond = [fips_long[i][:lcode] == code for i in np.arange(0,len(fips_long))]
    # Isolate only census tract plots in region
    if sum(cond) == 0:
        return '.','.'
    
    else:
        x1 = X[cond]
        y1 = Y[cond]
        z1 = Z[cond]
        # Take average
        x = np.average(x1)
        y = np.average(y1)
        z = np.average(z1)
        # convert average to lat and lon
        Lon = np.arctan2(y, x)
        Hyp = np.sqrt(x * x + y * y)
        Lat = np.arctan2(z, Hyp)
        return str(np.rad2deg(Lat)), str(np.rad2deg(Lon))

######################################################################
# Find lat and longitude for all FIPS county codes.
#
# Procedure:
#     - Use census tract plot data points within county
#         * If no tract plots given in census: lat, lon = '.'
#     - Convert to cartesian, average, convert to lat/lon
#  
#
# Runtime: about 10/15 minutes.  Likely inefficient but only runs once
######################################################################

average_lat = []
average_lon = []

for i in np.arange(0,len(fips_names)):
    code = fips_names[i,0]
    loc = avg_loc(code)
    average_lat.append(loc[0])
    average_lon.append(loc[1])
    if i%200 == 0:
        print(len(fips_names)-i)
        
        
######################################################################

# Create Dataframe stating country 
country = np.chararray((len(fips)), itemsize=10)
country[:] = 'US'
country = pd.DataFrame(country.decode('utf-8'),columns=['Country'])
# Make latitudes and longitudes into Pandas Dataframe
latitude_final = pd.DataFrame(average_lat, columns=['Lat'])
longitude_final = pd.DataFrame(average_lon, columns=['Lat'])
# Combine all columns
location_array = pd.concat([fips, country,latitude_final,longitude_final], axis=1)


############ Save to CSV file.  Previously unhashed ################
#location_array.to_csv('../FIPS_tags/locationkey_without_flag.csv',index=False)

3232


KeyboardInterrupt: 